# Day Two: Service Charge Enrichment

This notebook enriches BBF Service_Charge__c records with additional fields from ES OrderItem.

## Day 1 Fields (Already Migrated)
- Name, Service__c, mrc__c, nrc__c
- ES_Legacy_ID__c

## Day 2 Enrichment Fields
| BBF Field | ES Source | Transform |
|-----------|-----------|----------|
| Description__c | Description | Direct copy |
| Sequence__c | OrderItemNumber | Direct copy |
| Amount__c | Total_MRC_Amortized__c | Direct copy |
| Charge_Class__c | Charge_Class__c | Picklist map |
| Product_Simple__c | Product_Name__c | Picklist map |
| Service_Type_Charge__c | Product_Family__c | Picklist map |
| Quantity__c | Quantity | Direct copy |

In [ ]:
# === SETUP & IMPORTS ===

import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from datetime import datetime
from transformers.service_charge_transformers import TRANSFORMERS, FIELD_MAPPING, CHARGE_CLASS_MAP

print(f"Python: {sys.executable}")
print("Loaded service charge transformers")
print(f"  - {len(CHARGE_CLASS_MAP)} Charge Class mappings")
print("\n✅ Setup complete")

In [ ]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# BBF Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"

# Enrichment Options
DRY_RUN = True
TEST_LIMIT = 100

# Product mapping (ES Product_Name__c -> BBF Product_Simple__c)
# This needs business review - placeholder mappings
PRODUCT_SIMPLE_MAP = {
    'Dedicated Internet Access': 'DIA',
    'Point to Point': 'Transport',
    'Dark Fiber': 'Dark Fiber',
    'Wavelength': 'Wavelength',
    'Ethernet': 'Ethernet',
    'MPLS': 'MPLS',
    'Voice': 'Voice',
    'Colocation': 'Colocation',
    # Add more mappings as needed
}

# Output
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"service_charge_enrichment_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   DRY_RUN: {DRY_RUN}")
print(f"   TEST_LIMIT: {TEST_LIMIT}")

In [ ]:
# === CONNECT TO SALESFORCE ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

In [ ]:
# === QUERY MIGRATED RECORDS ===

print("\n" + "=" * 80)
print("QUERYING MIGRATED RECORDS")
print("=" * 80)

# Query ES OrderItem with enrichment fields
es_query = """
SELECT Id, BBF_New_Id__c,
       Description, OrderItemNumber,
       Total_MRC_Amortized__c, Charge_Class__c,
       Product_Name__c, Product_Family__c,
       Quantity, UnitPrice
FROM OrderItem
WHERE BBF_New_Id__c != null
"""

if TEST_LIMIT:
    es_query += f" LIMIT {TEST_LIMIT}"

print("\n📌 Querying ES OrderItem (enrichment fields)...")
es_result = es_sf.query_all(es_query)
es_records = es_result['records']
print(f"   Found {len(es_records)} migrated OrderItem records")

# Build lookup
es_lookup = {r['BBF_New_Id__c']: r for r in es_records}

# Query BBF Service_Charge__c current values
bbf_ids = list(es_lookup.keys())
bbf_records = []

print("\n📌 Querying BBF Service_Charge__c (current values)...")
chunk_size = 200
for i in range(0, len(bbf_ids), chunk_size):
    chunk = bbf_ids[i:i+chunk_size]
    ids_str = "','".join(chunk)
    
    bbf_query = f"""
    SELECT Id, ES_Legacy_ID__c, Name,
           Description__c, Sequence__c,
           Amount__c, Charge_Class__c,
           Product_Simple__c, Service_Type_Charge__c,
           Quantity__c
    FROM Service_Charge__c
    WHERE Id IN ('{ids_str}')
    """
    
    result = bbf_sf.query_all(bbf_query)
    bbf_records.extend(result['records'])

print(f"   Found {len(bbf_records)} Service_Charge__c records in BBF")

In [ ]:
# === BUILD ENRICHMENT UPDATES ===

print("\n" + "=" * 80)
print("BUILDING ENRICHMENT UPDATES")
print("=" * 80)

# Enrichment field mappings (BBF -> ES)
ENRICHMENT_MAPPING = {
    'Description__c': 'Description',
    'Sequence__c': 'OrderItemNumber',
    'Amount__c': 'Total_MRC_Amortized__c',
    'Charge_Class__c': 'Charge_Class__c',
    'Product_Simple__c': 'Product_Name__c',
    'Service_Type_Charge__c': 'Product_Family__c',
    'Quantity__c': 'Quantity',
}

updates = []
update_details = []
field_stats = {field: {'enriched': 0, 'already_set': 0, 'no_source': 0} for field in ENRICHMENT_MAPPING}

# Track unmapped product values
unmapped_products = set()

for bbf_rec in bbf_records:
    bbf_id = bbf_rec['Id']
    
    if bbf_id not in es_lookup:
        continue
    es_rec = es_lookup[bbf_id]
    
    update_rec = {'Id': bbf_id}
    rec_details = {'bbf_id': bbf_id, 'name': bbf_rec.get('Name', 'N/A'), 'fields': []}
    
    for bbf_field, es_field in ENRICHMENT_MAPPING.items():
        bbf_value = bbf_rec.get(bbf_field)
        es_value = es_rec.get(es_field)
        
        if bbf_value:
            field_stats[bbf_field]['already_set'] += 1
        elif es_value:
            # Apply transformation
            if bbf_field == 'Charge_Class__c':
                transformed = CHARGE_CLASS_MAP.get(es_value, es_value)
            elif bbf_field == 'Product_Simple__c':
                transformed = PRODUCT_SIMPLE_MAP.get(es_value)
                if not transformed:
                    unmapped_products.add(es_value)
                    continue  # Skip unmapped products
            else:
                transformed = es_value
            
            if transformed:
                update_rec[bbf_field] = transformed
                rec_details['fields'].append({
                    'field': bbf_field,
                    'old': bbf_value,
                    'new': transformed,
                    'source': es_value
                })
                field_stats[bbf_field]['enriched'] += 1
        else:
            field_stats[bbf_field]['no_source'] += 1
    
    if len(update_rec) > 1:
        updates.append(update_rec)
        update_details.append(rec_details)

print(f"\n📊 Enrichment Analysis:")
print(f"   Total BBF records analyzed: {len(bbf_records)}")
print(f"   Records needing updates: {len(updates)}")
print(f"\n   Field Statistics:")
print(f"   {'Field':<30} | {'Enriched':>10} | {'Already Set':>12} | {'No Source':>10}")
print(f"   {'-'*70}")
for field, stats in field_stats.items():
    print(f"   {field:<30} | {stats['enriched']:>10} | {stats['already_set']:>12} | {stats['no_source']:>10}")

if unmapped_products:
    print(f"\n⚠️  Unmapped Product_Name__c values (need mapping):")
    for prod in sorted(unmapped_products):
        print(f"     - {prod}")

In [ ]:
# === APPLY ENRICHMENT UPDATES ===

print("\n" + "=" * 80)
print("APPLYING ENRICHMENT UPDATES")
print("=" * 80)

if len(updates) == 0:
    print("\n⚠️  No updates to apply")
    update_results = []
elif DRY_RUN:
    print(f"\n🔍 DRY RUN - Would update {len(updates)} Service_Charge__c records")
    print("\nSample updates (first 5):")
    for i, detail in enumerate(update_details[:5], 1):
        print(f"\n{i}. {detail['name']} ({detail['bbf_id'][:15]}...)")
        for f in detail['fields'][:3]:
            print(f"   {f['field']}: {f['old']} -> {f['new']}")
    update_results = [{'success': True, 'id': u['Id']} for u in updates]
else:
    print(f"\n📌 Updating {len(updates)} Service_Charge__c records...")
    
    try:
        update_results = bbf_sf.bulk.Service_Charge__c.update(updates)
        
        success_count = sum(1 for r in update_results if r['success'])
        error_count = sum(1 for r in update_results if not r['success'])
        
        print(f"\n✅ Successfully updated: {success_count}")
        print(f"❌ Failed to update: {error_count}")
    except Exception as e:
        print(f"\n❌ Error during update: {e}")
        update_results = []

In [ ]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()

ws1 = wb.active
ws1.title = "Summary"
ws1.append(["Service Charge Enrichment Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE UPDATE"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append(["Records Analyzed:", len(bbf_records)])
ws1.append(["Records Updated:", len(updates)])
ws1.append([])
ws1.append(["Field", "Enriched", "Already Set", "No Source"])
for field, stats in field_stats.items():
    ws1.append([field, stats['enriched'], stats['already_set'], stats['no_source']])

# Unmapped products sheet
if unmapped_products:
    ws1.append([])
    ws1.append(["Unmapped Product_Name__c values:"])
    for prod in sorted(unmapped_products):
        ws1.append([prod])

ws2 = wb.create_sheet("Update Details")
ws2.append(["BBF Service Charge ID", "Name", "Field", "Old Value", "New Value", "ES Source"])
for detail in update_details:
    for f in detail['fields']:
        ws2.append([detail['bbf_id'], detail['name'], f['field'], str(f['old'] or ''), str(f['new']), str(f['source'])])

wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")

In [ ]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("SERVICE CHARGE ENRICHMENT COMPLETE")
print("=" * 80)
print(f"\nRecords analyzed: {len(bbf_records)}")
print(f"Records {'would be ' if DRY_RUN else ''}updated: {len(updates)}")
print(f"Output file: {output_file}")

if unmapped_products:
    print(f"\n⚠️  {len(unmapped_products)} Product_Name__c values need mapping")

if DRY_RUN:
    print("\n⚠️  This was a DRY RUN - no changes were made")